In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
sys.path.append('../') 
from reclab.environments.topics import Topics
from reclab.environments.latent_factors import LatentFactorBehavior
from reclab.recommenders.libfm.libfm import LibFM
import matplotlib.pyplot as plt

# Illustration of Different User Behaviors

In this notebook, for a fixed level of model and user behavior complexity, we illustrate the effect of different time-varying user behaviors on recommendation system quality. 
Specifically, we evaluate two metrics: observed ratings of recommended items, and predictive accuracy on recommended items. We may also want to look at diversity metrics, or those used in other papers.

In [ ]:
## Key Parameters across all settings

num_users = 100
num_items = 170
rating_frequency = 0.2
num_init_ratings = 1000

n_trials = 1 # 0
len_trial = 2 # 0

SEED = 24532

topics = False


## Main experimentation functionality - only use environment changes
def run_experiment(env):
    all_mean_ratings = []; all_mses = []
    for i in range(n_trials):
        mean_ratings, mses = run_trial(env)
        all_mean_ratings.append(mean_ratings)
        all_mses.append(mses)
    return np.array(all_mean_ratings), np.array(all_mses)

def run_trial(env):
    recommender = LibFM(num_user_features=0, num_item_features=0, 
                        num_rating_features=0, max_num_users=num_users, 
                        max_num_items=num_items)

    # First generate the items and users to seed the dataset.
    print("Initializing environment and recommender")
    items, users, ratings = env.reset()
    recommender.reset(items, users, ratings)

    mean_ratings = []
    mses = []
    # Now recommend items to users.
    print("Making online recommendations")
    for i in range(len_trial):
        online_users = env.online_users()
        ret, predicted_ratings = recommender.recommend(online_users, num_recommendations=1)
        recommendations = ret[:, 0]
        items, users, ratings, info = env.step(recommendations)
        recommender.update(users, items, ratings)
        rating_arr = []
        for (rating, _), pred in zip(ratings.values(), predicted_ratings):
            rating_arr.append([rating, pred])
        rating_arr = np.array(rating_arr)
        errors = rating_arr[:,0] - rating_arr[:,1]
        mean_ratings.append(np.mean(rating_arr[:, 0]))
        mses.append(np.mean(errors**2))
        print("Iter:", i, "Mean:", mean_ratings[-1], "MSE:", mses[-1])

    ratings = env.all_ratings()
    return mean_ratings, mses

## Static User Behavior

Here, the user model does not change over time. 

In [ ]:
if topics:
    env = Topics(num_topics=10, num_users=num_users, num_items=num_items, 
             num_init_ratings=num_init_ratings, rating_frequency=rating_frequency)
else:
    env = LatentFactorBehavior(latent_dim=8, num_users=num_users, num_items=num_items, 
             num_init_ratings=num_init_ratings, rating_frequency=rating_frequency)
env.seed(SEED)
mean_ratings, mses = run_experiment(env)

In [ ]:
plt.figure(figsize=[9,4])
plt.subplot(1,2,1)
xs = num_init_ratings + num_users/rating_frequency * np.arange(mean_ratings.shape[1])
plt.plot(xs, np.mean(mean_ratings, axis=0), label='mean rating')
plt.xlabel('# ratings'); plt.ylabel('mean rating')
plt.subplot(1,2,2)
plt.plot(xs, np.mean(mses, axis=0), label='mse')
plt.xlabel('# ratings'); plt.ylabel('mse')
plt.tight_layout()
plt.show()


## Partially Hidden User Behavior

Here, there is a portion of item utility that is unknown to a user until the item is consumed. From the paper "How Algorithmic Confounding in Recommendation Systems
Increases Homogeneity and Decreases Utility".

## User Interests Shift with Exposure

Here, the underlying user interest changes over time to match the body of items that have been recommended.

In [ ]:
if topics:
    env = Topics(num_topics=10, num_users=num_users, num_items=num_items, 
             num_init_ratings=num_init_ratings, rating_frequency=rating_frequency,
             topic_change=0.1)
else:
    env = LatentFactorBehavior(latent_dim=8, num_users=num_users, num_items=num_items, 
             num_init_ratings=num_init_ratings, rating_frequency=rating_frequency,
             affinity_change=0.1)

env.seed(SEED)

mean_ratings, mses = run_experiment(env)

In [ ]:
plt.figure(figsize=[9,4])
plt.subplot(1,2,1)
xs = num_init_ratings + num_users/rating_frequency * np.arange(mean_ratings.shape[1])
plt.plot(xs, np.mean(mean_ratings, axis=0), label='mean rating')
plt.xlabel('# ratings'); plt.ylabel('mean rating')
plt.subplot(1,2,2)
plt.plot(xs, np.mean(mses, axis=0), label='mse')
plt.xlabel('# ratings'); plt.ylabel('mse')
plt.tight_layout()
plt.show()


## User Gets Bored

Here, user interest decreases for items too similar to those that they have recently been exposed to, with some decay in the effect.

In [ ]:
if topics:
    env = Topics(num_topics=10, num_users=num_users, num_items=num_items, 
             num_init_ratings=num_init_ratings, rating_frequency=rating_frequency,
             memory_length=5, boredom_threshold=2, boredom_penalty=1.0)
else:
    env = LatentFactorBehavior(latent_dim=8, num_users=num_users, num_items=num_items, 
             num_init_ratings=num_init_ratings, rating_frequency=rating_frequency,
             memory_length=5, boredom_threshold=0.5, boredom_penalty=1.0)

env.seed(SEED)

mean_ratings, mses = run_experiment(env)

In [ ]:
plt.figure(figsize=[9,4])
plt.subplot(1,2,1)
xs = num_init_ratings + num_users/rating_frequency * np.arange(mean_ratings.shape[1])
plt.plot(xs, np.mean(mean_ratings, axis=0), label='mean rating')
plt.xlabel('# ratings'); plt.ylabel('mean rating')
plt.subplot(1,2,2)
plt.plot(xs, np.mean(mses, axis=0), label='mse')
plt.xlabel('# ratings'); plt.ylabel('mse')
plt.tight_layout()
plt.show()


## User Interests Shift, also Gets Bored

This is a combination of the previous two cases, where users gradually become more interested in topics that they are exposed to, but also exhibit boredom when they see many similar items in a row.

In [ ]:
if topics:
    env = Topics(num_topics=10, num_users=num_users, num_items=num_items, 
             num_init_ratings=num_init_ratings, rating_frequency=rating_frequency,
             topic_change=0.1, memory_length=5, boredom_threshold=2, boredom_penalty=1.0)
else:
    env = LatentFactorBehavior(latent_dim=8, num_users=num_users, num_items=num_items, 
             num_init_ratings=num_init_ratings, rating_frequency=rating_frequency,
             affinity_change=0.1, memory_length=5, boredom_threshold=2, boredom_penalty=1.0)

env.seed(SEED)

mean_ratings, mses = run_experiment(env)

In [ ]:
plt.figure(figsize=[9,4])
plt.subplot(1,2,1)
xs = num_init_ratings + num_users/rating_frequency * np.arange(mean_ratings.shape[1])
plt.plot(xs, np.mean(mean_ratings, axis=0), label='mean rating')
plt.xlabel('# ratings'); plt.ylabel('mean rating')
plt.subplot(1,2,2)
plt.plot(xs, np.mean(mses, axis=0), label='mse')
plt.xlabel('# ratings'); plt.ylabel('mse')
plt.tight_layout()
plt.show()
